In [1]:
import torch
import pandas as pd
import numpy as np
import numpy.random as npr
import itertools
# from torchvision.datasets import CIFAR10
# from torchvision.transforms import Compose, RandomCrop, RandomHorizontalFlip, CenterCrop

### Number of Quantization Categories

In [2]:
def simulate_probably(n, k, b, s=1000):
    np.random.seed(123)
    count = 0
    for _ in range(s):
        objects = np.random.choice(k, size=(n,))
        batch = np.random.choice(objects, size=(b,), replace=False)
        count += int(len(np.unique(batch)) == k)
    return count / s

In [3]:
n = 50000
b = 512
k = 60

simulate_probably(n, k, b)

0.995

### Raking Estimator

In [4]:
def is_good_event(ix, iy, px, py):
    return len(torch.unique(ix)) == len(px) and len(torch.unique(iy)) == len(py)

def get_empirical_pmf(ix, iy):
    # count pairs
    pairs = list(zip(ix, iy))
    ind, count = np.unique(pairs, axis=0, return_counts=True)
    cmat = np.zeros(sizes)
    cmat[ind[:, 0], ind[:, 1]] = count
    return cmat / len(pairs)

def raking_ratio(X, Y, marginals, num_iter):
    pmat = count_freq(X, Y, (len(marginals[0]), len(marginals[1])))
    if np.sum(np.sum(pmat, axis=1) == 0) + np.sum(np.sum(pmat, axis=0) == 0) > 0:
        raise RuntimeError(
            "Missing mass in this sample. Try a larger sample size.")
        
    est = [pmat]
    for _ in range(num_iter):
        pmat = (marginals[0] / np.sum(pmat, axis=1)).reshape(-1, 1) * pmat
        pmat = pmat * (marginals[1] / np.sum(pmat, axis=0))
        est.append(pmat)
    return est

In [5]:
def generate(pmat, size):
    pairs = np.argwhere(pmat > -1)
    indices = npr.choice(len(pairs), size=size, p=pmat.reshape(-1), replace=True)
    return pairs[indices][:, 0], pairs[indices][:, 1]

def count_freq(X, Y, sizes):
    # count pairs
    pairs = list(zip(X, Y))
    ind, count = np.unique(pairs, axis=0, return_counts=True)
    cmat = np.zeros(sizes)
    cmat[ind[:, 0], ind[:, 1]] = count
    return cmat / len(pairs), ind, count

In [6]:
npr.seed(552022)  # 05/05/2022

m = 20
prob = npr.rand(m, m)
prob /= np.sum(prob)
px, py = np.sum(prob, axis=1), np.sum(prob, axis=0)

In [7]:
marginals = (px, py)
n = 512
num_iter = 2
X, Y = generate(prob, n)

pmat, observed_bins, observed_counts = count_freq(X, Y, (len(marginals[0]), len(marginals[1])))
# if np.sum(np.sum(pmat, axis=1) == 0) + np.sum(np.sum(pmat, axis=0) == 0) > 0:
#      raise RuntimeError("Missing mass in this sample. Try a larger sample size.")


In [8]:
pmat.shape

(20, 20)

In [10]:
df1 = pd.DataFrame(
    {
        'ind': list(itertools.product(np.arange(n), np.arange(n))),
        'bins': list(itertools.product(X, Y)),
    }
)
df2 = pd.DataFrame(
    {
        'bins': list(itertools.product(np.arange(m), np.arange(m))),
        'prob': pmat.reshape(-1)
    }
)
bin_names, bin_counts = np.unique(df1['bins'], return_counts=True)
# df3 = pd.DataFrame({'bins': [tuple(x) for x in observed_bins], "counts": observed_counts})
# df = df1.merge(df2.merge(df3, on="bins", how="left").fillna(0.0), on="bins", how="left")
df = df1.merge(df2, on="bins", how="left").merge(pd.DataFrame({'bins': bin_names, 'counts': bin_counts}), on="bins", how="left")
weight = df['prob'].to_numpy() / np.maximum(1.0, df['counts'].to_numpy())
df

,ind,bins,prob,counts
0,"(0, 0)","(7, 8)",0.005859,726
1,"(0, 1)","(7, 17)",0.000000,660
2,"(0, 2)","(7, 16)",0.003906,792
3,"(0, 3)","(7, 10)",0.007812,1023
4,"(0, 4)","(7, 4)",0.000000,759
...,...,...,...,...
262139,"(511, 507)","(3, 12)",0.001953,783
262140,"(511, 508)","(3, 18)",0.000000,567
262141,"(511, 509)","(3, 4)",0.001953,621
262142,"(511, 510)","(3, 10)",0.005859,837


In [12]:
weight.sum()

1.0

In [13]:
idx = np.random.choice(np.arange(n ** 2), size=n, replace=True, p=weight)

In [16]:
new_pairs = df['ind'].iloc[idx]

In [18]:
x_new, y_new = [X[i] for i, j in new_pairs], [Y[j] for i, j in new_pairs]

In [20]:
len(x_new)

512

In [ ]:
[tuple(x) for x in bins]

In [ ]:
len(counts)